# L-Alanin with GPAW

This is an xHARPy example file for demonstrating how a Hirshfeld Atom Refinement with GPAW is set up.

In [1]:
from xharpy import shelxl_hkl_to_pd, cif2data, lst2constraint_dict, create_construction_instructions, refine, write_cif, write_res, write_fcf
import os

In [2]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [3]:
atom_table, cell, cell_esd, symm_mats_vecs, wavelength, symm_strings = cif2data('iam.cif', 0)

In [4]:
hkl = shelxl_hkl_to_pd('iam.hkl')

In [5]:
constraint_dict = lst2constraint_dict(
    'iam.lst'
)

In [9]:
refinement_dict = {
    'f0j_source': 'gpaw',
    'reload_step': 1,
    'core': 'constant',
    'save_file': os.path.join(output_folder, 'gpaw_result.gpw')
}

In [10]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)


In [14]:
computation_dict = {
    'xc': 'SCAN',
    'txt': os.path.join(output_folder, 'gpaw.txt'),
    'mode': 'fd',
    'basis': 'tzp',
    'h': 0.15,
    'gridinterpolation': 4,
    #'poissonsolver': poissonsolver,d
    'average_symmequiv': False,
    'convergence':{'density': 1e-7},
    'kpts': {'size': (2, 2, 2), 'gamma': True},
    #'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2
}

In [ ]:
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-03 16:20:01.641493
Preparing
  calculating the core structure factor for O


In [8]:
?refine

Signature:
refine(
    cell: jax._src.numpy.lax_numpy.ndarray,
    symm_mats_vecs: Tuple[jax._src.numpy.lax_numpy.ndarray, jax._src.numpy.lax_numpy.ndarray],
    hkl: pandas.core.frame.DataFrame,
    construction_instructions: List[xharpy.core.AtomInstructions],
    parameters: jax._src.numpy.lax_numpy.ndarray,
    wavelength: Union[float, NoneType] = None,
    refinement_dict: dict = {},
    computation_dict: dict = {},
) -> Tuple[jax._src.numpy.lax_numpy.ndarray, jax._src.numpy.lax_numpy.ndarray, Dict[str, Any]]
Docstring:
Refinement routine. The routine will refine for the given intensities
against wR2(F^2).

Parameters
----------
cell : jnp.ndarray
    array with the lattice constants (Angstroem, Degree)
symm_mats_vecs : Tuple[jnp.ndarray, jnp.ndarray]
  size (K, 3, 3) array of symmetry matrices and (K, 3) array of translation
    vectors for all symmetry elements in the unit cell
hkl : pd.DataFrame
    pandas DataFrame containing the reflection data. Needs to have at least
    fiv